In [1]:
import pandas as pd

df = pd.read_csv('/Users/patwilliams/DSCI441/DSCI411-project/Data_PreProcessing/final_economic_data.csv')

In [2]:
df.head()

,Year,Annual_%_Change_nasdaq,Annual_%_Change_dowj,Annual_%_Change_sp500,Annual_%_Change_corn,Annual_%_Change_cotton,Annual_%_Change_pound,Annual_%_Change_yen,Annual_Unemployment_Rate,Annual_%_Change_silver,Annual_%_Change_sugar,Annual_%_Change_FFRate,Annual_%_Change_tenyrRate,Annual_%_Change_oneyrRate,Annual_%_Change_gold,Annual_%_Change_crude,Annual_%_Change_copper,Annual_%_Change_coffee
0,2024.0,9.05,5.55,9.89,-6.53,13.58,-0.51,6.85,3.800000,-0.092688,-0.048746,-0.223298,0.262182,-0.145917,-0.144743,0.056138,-0.134274,-0.345731
1,2023.0,43.42,13.70,24.23,-30.62,-2.41,5.22,7.56,3.625000,-0.229543,-0.148293,-0.070694,-0.082794,-0.199847,0.188286,-0.582778,-0.166927,0.208869
2,2022.0,-33.10,-8.78,-19.44,14.37,-26.29,-10.52,13.91,3.633333,-0.124077,-0.059586,6.297961,3.488887,3.026747,-0.340666,-0.128344,-0.646130,-0.838549
3,2021.0,21.39,18.73,26.89,22.57,44.14,-1.23,11.49,5.341667,-0.569481,0.287502,-0.370152,2.094849,3.026747,-0.471016,1.052936,0.566733,1.624098
4,2020.0,43.64,7.25,16.26,24.82,13.14,3.21,-5.00,8.091667,1.282133,0.144376,-0.845809,-1.852644,-1.742681,0.639346,-0.807901,0.536434,-0.237071


In [3]:
from sklearn.model_selection import train_test_split

# Assuming the DataFrame is already sorted by year
# Extract the row for the holdout set (the most recent year, 2024)
holdout = df[df['Year'] == 2024]

# Remove the holdout set from the main DataFrame
df_modeling = df[df['Year'] != 2024]

# IF YEAR SHOULD BE INCORPORATED

# Calculate 'Years Since Epoch' where Epoch is set to 1974
#df['Years_Since_Epoch'] = df['Year'] - 1974
# Display the updated DataFrame to verify the transformation
#df[['Year', 'Years_Since_Epoch']].head()


# Define the features and target
X = df_modeling.drop(['Annual_Unemployment_Rate', 'Year'], axis=1) # should year be included?? Is it better to include as t starting with t = 0... t+1
y = df_modeling['Annual_Unemployment_Rate']

# Split the data into training and testing sets
# We want 40 observations in the training set, and 10 in the test set
# The dataset is already sorted by year, so we can split accordingly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10, shuffle=False)

# Check the shapes to confirm the sizes
(X_train.shape, X_test.shape, y_train.shape, y_test.shape, holdout.shape)

((40, 16), (10, 16), (40,), (10,), (1, 18))

In [4]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor

reg_ada = AdaBoostRegressor(n_estimators=10, random_state=500)
reg_ada.fit(X_train, y_train)

# Calculate the predictions on the test set
pred = reg_ada.predict(X_test)

# Evaluate the performance using the RMSE
rmse = np.sqrt(mean_squared_error(y_test, pred))
print('RMSE: {:.3f}'.format(rmse))

RMSE: 2.923


In [5]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [10, 50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.5, 1.0]

}

# Initialize the AdaBoostRegressor
reg_ada = AdaBoostRegressor(random_state=500)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=reg_ada, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameter and score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation RMSE:", np.sqrt(-grid_search.best_score_))

Best parameters: {'learning_rate': 0.01, 'n_estimators': 100}
Best cross-validation RMSE: 1.8046500885653922


In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
best_ada = grid_search.best_estimator_

# Make predictions using the best model on the test set
pred = best_ada.predict(X_test)

# Calculate various performance metrics
mse = mean_squared_error(y_test, pred)
mae = mean_absolute_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)

# Print the performance metrics
print("Best Model Performance Metrics:")
print("Mean Squared Error (MSE): {:.3f}".format(mse))
print("Mean Absolute Error (MAE): {:.3f}".format(mae))
print("Root Mean Squared Error (RMSE): {:.3f}".format(rmse))
print("R-squared (R2): {:.3f}".format(r2))

Best Model Performance Metrics:
Mean Squared Error (MSE): 6.045
Mean Absolute Error (MAE): 2.057
Root Mean Squared Error (RMSE): 2.459
R-squared (R2): -2.211
